In [ ]:
import math
import numpy as np
import pandas as pd
from numba import *
from numba import jit, guvectorize
from numba import cuda, float32
import matplotlib.pyplot as plt
import scipy.spatial.distance as spd
from timeit import default_timer as timer

In [ ]:
ex_1k = np.random.rand(1024, 1024)
ex_2k = np.random.rand(2048, 2048)
ex_4k = np.random.rand(4096, 4096)
ex_10k = np.random.rand(10240, 10240)
# ex_20k = np.random.rand(20480, 20480)
# ex_40k = np.random.rand(40960, 40960)

In [ ]:
ex_16 = np.random.rand(16, 16)

# Linear kernel

In [ ]:
def np_linear_kernel(X, Y):
    return X @ Y.T

In [ ]:
%%time
np_linear_kernel(ex_10k, ex_10k)

In [ ]:
@jit(nopython=True, parallel=True)
def jit_linear_kernel(X, Y):
    return X @ Y.T

In [ ]:
%%time
jit_linear_kernel(ex_4k, ex_4k.T)

In [ ]:
# @jit("float32(f4[:,:], f4[:,:])")
# def jit_linear_kernel(x, y):
#     c = np.empty([x.shape[0], y.shape[1]])
#     for i in range(x.shape[0]):
#         for j in range(y.shape[1]):
#             for k in range(x.shape[1]):
#                 c[i, j] += x[i, k] * y[k, j]
#     return c

In [ ]:
# @jit("void(float32[:,:],float32[:,:],float32[:,:])")
# def jit_linear_kernel(x, y, c):
#     for i in range(x.shape[0]):
#         for j in range(y.shape[1]):
#             for k in range(x.shape[1]):
#                 c[i, j] += x[i, k] * y[k, j]
#     return c

In [ ]:
cuda.select_device(0)

In [ ]:
gpu = cuda.get_current_device()
gpu.name

In [ ]:
TPB = 32

# source: https://stackoverflow.com/questions/64197780/how-to-generalize-fast-matrix-multiplication-on-gpu-using-numba
@cuda.jit
def cuda_linear_kernel(A, B, C):
    
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = float32(0.)
    
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = 0
        sB[tx, ty] = 0
        
        # Verifying that all shapes are properly respected for matrix A and B
        if x < A.shape[0] and (ty + i * TPB) < A.shape[1]:
#             sA[tx, ty] = A[x, ty + i * TPB]
            sA[ty, tx] = A[x, ty + i * TPB]
            
        if y < B.shape[1] and (tx +i * TPB) < B.shape[0]:
#             sB[tx, ty] = B[tx + i * TPB, y]
            sB[ty, tx] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
#             tmp += sA[tx, j] * sB[j, ty]
            tmp += sA[ty, j] * sB[j, tx]
            

        # Wait until all threads finish computing
        cuda.syncthreads()
        
    if x < C.shape[0] and y < C.shape[1]:
        C[x, y] = tmp

In [ ]:
time_linear_np = []
time_linear_jit = []
time_linear_cuda = []

for arr in [ex_1k, ex_2k, ex_4k, ex_10k]:
    
    start = timer()
    test = np_linear_kernel(arr, arr)
    end_np = timer() - start
    time_linear_np.append(end_np)
    
    # jit version
    c = np.empty([arr.shape[0], arr.shape[1]])
    start = timer()
    test = jit_linear_kernel(arr, arr.T)
    end_py = timer() - start
    time_linear_jit.append(end_py)
    
    # cuda version
    x_h = arr
    y_h = arr
    z_h = np.zeros([arr.shape[0], arr.shape[1]])


    x_d = cuda.to_device(x_h)
    y_d = cuda.to_device(y_h)
    z_d = cuda.to_device(z_h)

    TPB = 32
    threadsperblock = (TPB, TPB)
    blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
    blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    
    start = timer()
    test = cuda_linear_kernel[blockspergrid, threadsperblock](x_d, y_d, z_d)
#     z_h = z_d.copy_to_host()
    end_cuda = timer() - start
    time_linear_cuda.append(end_cuda)

In [ ]:
z_h

In [ ]:
z_h2

In [ ]:
size_arr = ["1024", "2048", "4096", "10240"]
linear_df = pd.DataFrame({"numpy": time_linear_np, "jit": time_linear_jit, "cuda": time_linear_cuda}, 
                         index=size_arr)
linear_df.plot.bar(rot=0)
plt.title("linear kernel perf. based on matrix size")
plt.xlabel("matrix size")
plt.ylabel("time (s)");
plt.savefig("linear_performance.png")

In [ ]:
# extracting results pandas data frame
# linear_df.to_csv("./linear_df.csv")

In [ ]:
speed_up_linear_df = linear_df.apply(lambda x: linear_df['numpy'] / x)

In [ ]:
speed_up_linear_df

In [ ]:
speed_up_linear_df.plot.bar(rot=0)
plt.title("speedup vs numpy version for linear kernel")
plt.xlabel("matrix size")
plt.ylabel("speedup");
plt.savefig("linear_speedup.png")

# Polynomial kernel

In [ ]:
def np_polynomial_kernel(X, Y, c, r):
    """Numpy version of the polynomial kernel"""
    return (X @ Y.T + c) ** r

In [ ]:
%%time
np_polynomial_kernel(ex_10k, ex_10k, 2., 2.)

In [ ]:
@jit(nopython=True, parallel=True)
def np_jit_polynomial_kernel(X, Y, c, r):
    """Numpy version of the polynomial kernel"""
    return (X @ Y.T + c) ** r

In [ ]:
%%time
np_jit_polynomial_kernel(ex_10k, ex_10k, 2., 2.)

In [ ]:
@jit("void(float32[:,:], float32[:,:], float32[:,:])")
def jit_polynomial_kernel(X, Y, C):
    """Just in time version of the polynomial kernel"""
#     C = np.zeros([X.shape[0], Y.shape[1]])
    for i in range(X.shape[0]): 
        for j in range(Y.shape[1]):      
            for k in range(X.shape[1]):
                C[i, j] += (X[i, k] * Y[k, j])
                
    C = (C + 2.)**2.
    
    return C

In [ ]:
%%time
C = np.zeros([ex_1k.shape[0], ex_1k.shape[1]])
jit_polynomial_kernel(ex_1k, ex_1k, C)

In [ ]:
TPB = 32

@cuda.jit
def cuda_polynomial_kernel(A, B, C, c=2., r=2.):
    """Cuda version of the polynomial kernel"""

    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    tmp = float32(0.)
    
    for i in range(bpg):
        
        sA[tx, ty] = 0
        sB[tx, ty] = 0
        
        if x < A.shape[0] and (ty + i * TPB) < A.shape[1]:
            sA[tx, ty] = A[x, ty + i * TPB]
            
        if y < B.shape[1] and (tx + i * TPB) < B.shape[0]:
            sB[tx, ty] = B[tx + i * TPB, y]

        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += (sA[tx, j] * sB[j, ty])

        cuda.syncthreads()
        
    if x < C.shape[0] and y < C.shape[1]:
        C[x, y] = (tmp + c)**2

In [ ]:
# cuda version
x_h = ex_1k
y_h = ex_1k
z_h = np.zeros_like(ex_1k)


x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)

TPB = 32
threadsperblock = (TPB, TPB)
blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

start = timer()
test = cuda_polynomial_kernel[blockspergrid, threadsperblock](x_d, y_d, z_d, 2., 2.)

In [ ]:
z_h = z_d.copy_to_host()
z_h

In [ ]:
time_poly_np = []
time_poly_jit = []
time_poly_cuda = []

for arr in [ex_1k, ex_2k, ex_4k, ex_10k]:
    
    start = timer()
    test = np_polynomial_kernel(arr, arr, 2., 2.)
    end_np = timer() - start
    time_poly_np.append(end_np)
    
    # jit version
    C = np.zeros([arr.shape[0], arr.shape[1]])
    start = timer()
    test = np_jit_polynomial_kernel(arr, arr, 2., 2.)
    end_py = timer() - start
    time_poly_jit.append(end_py)
    
    # cuda version
    x_h = arr
    y_h = arr
    z_h = np.zeros([arr.shape[0], arr.shape[1]])


    x_d = cuda.to_device(x_h)
    y_d = cuda.to_device(y_h)
    z_d = cuda.to_device(z_h)

    TPB = 32
    threadsperblock = (TPB, TPB)
    blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
    blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    
    start = timer()
    test = cuda_polynomial_kernel[blockspergrid, threadsperblock](x_d, y_d, z_d, 2., 2.)
#     z_h = z_d.copy_to_host()
    end_cuda = timer() - start
    time_poly_cuda.append(end_cuda)    

In [ ]:
size_arr = ["1024", "2048", "4096", "10240"]
poly_df = pd.DataFrame({"numpy": time_poly_np, "jit": time_poly_jit, "cuda": time_poly_cuda}, 
                         index=size_arr)
poly_df.plot.bar(rot=0)
plt.title("polynomial kernel perf. based on matrix size")
plt.xlabel("matrix size")
plt.ylabel("time (s)");
plt.savefig("polynomial_performance.png")

In [ ]:
speed_up_poly_df = poly_df.apply(lambda x: poly_df['numpy'] / x)

In [ ]:
# extracting results pandas data frame
poly_df.to_csv("./poly_df.csv")

In [ ]:
speed_up_poly_df.plot.bar(rot=0)
plt.title("speedup vs numpy version for polynomial kernel")
plt.xlabel("matrix size")
plt.ylabel("speedup");
plt.savefig("polynomial_speedup.png")

# Gaussian kernel

In [ ]:
def np_gaussian_kernel(X, Y, sigma = 1.):
    """Numpy (and scipy) version of the gaussian kernel"""
    # same matrices
    if np.array_equal(X, Y) and X.ndim > 1:
        return np.exp( - spd.squareform(spd.pdist(X, 'euclidean'))**2 / (2*sigma**2) )

    elif not np.array_equal(X, Y) and X.shape[0] != Y.shape[0] and X.shape[1] == Y.shape[1]: # two matrices of different sizes
        return np.exp( - spd.cdist(X, Y, 'euclidean')**2/ (2*sigma**2))

    elif not np.array_equal(X, Y) and X.ndim == Y.ndim == 1: # vectors
        return np.exp(- np.linalg.norm(X - Y)**2 / (2*sigma**2))

In [ ]:
%%time
np_gaussian_kernel(ex_10k, ex_10k)

In [ ]:
@jit
def jit_gaussian_kernel(X, Y, sigma=1.):
    C = np.empty([X.shape[0], Y.shape[1]])
    for i in range(X.shape[0]):
        C[i, :] = (X[i,:] - Y[:, i])**2
        C[i, :] = np.exp(-C[i, :] / (2*sigma**2))
    return C

In [ ]:
%%time
jit_gaussian_kernel(ex_10k, ex_10k)

In [ ]:
TPB = 32

@cuda.jit
def cuda_gaussian_kernel(A, B, C, sigma=1.):
    
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    bpg = cuda.gridDim.x    # blocks per grid

    tmp = float32(0.)
    for i in range(bpg):
        
        sA[tx, ty] = 0
        sB[tx, ty] = 0
        
        if x < A.shape[0] and (ty + i * TPB) < A.shape[1]:
            sA[tx, ty] = A[x, ty + i * TPB]
            
        if y < B.shape[1] and (tx+i*TPB) < B.shape[0]:
            sB[tx, ty] = B[tx + i * TPB, y]

        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += (sA[tx, j] - sB[j, ty])**2
#             tmp = math.exp(-tmp / 2 * sigma**2)
#             tmp += math.exp(-((sA[tx, j] - sB[j, ty])**2)/2*sigma**2) 

        cuda.syncthreads()
        
        tmp = math.exp(-tmp / (2 * sigma**2))
        
    if x < C.shape[0] and y < C.shape[1]:
        C[x, y] = tmp
#         C[x, y] = math.exp(-C[x, y] / (2 * sigma**2))

In [ ]:
time_np = []
time_jit = []
time_cuda = []

for arr in [ex_1k, ex_2k, ex_4k, ex_10k]:
    
    start = timer()
    test = np_gaussian_kernel(arr, arr)
    end_np = timer() - start
    time_np.append(end_np)
    
    # jit version
    start = timer()
    test = jit_gaussian_kernel(arr, arr)
    end_py = timer() - start
    time_jit.append(end_py)
    
    # cuda version
    x_h = arr
    y_h = arr
    z_h = np.zeros([arr.shape[0], arr.shape[1]])


    x_d = cuda.to_device(x_h)
    y_d = cuda.to_device(y_h)
    z_d = cuda.to_device(z_h)

    TPB = 32
    threadsperblock = (TPB, TPB)
    blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
    blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    
    start = timer()
    test = cuda_gaussian_kernel[blockspergrid, threadsperblock](x_d, y_d, z_d, 1.)
    #z_h = z_d.copy_to_host()
    end_cuda = timer() - start
    time_cuda.append(end_cuda)    

In [ ]:
# cuda version
# x_h = ex_1k
# y_h = ex_1k
# z_h = np.zeros_like(ex_1k)


# x_d = cuda.to_device(x_h)
# y_d = cuda.to_device(y_h)
# z_d = cuda.to_device(z_h)

# TPB = 32
# threadsperblock = (TPB, TPB)
# blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
# blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
# blockspergrid = (blockspergrid_x, blockspergrid_y)

# start = timer()
# test = cuda_gaussian_kernel[blockspergrid, threadsperblock](x_d, y_d, z_d, 1.)

In [ ]:
# z_h = z_d.copy_to_host()
# z_h

In [ ]:
size_arr = ["1024", "2048", "4096", "10240"]
gauss_df = pd.DataFrame({"numpy": time_np, "jit": time_jit, "cuda": time_cuda}, 
                         index=size_arr)
gauss_df.plot.bar(rot=0)
plt.title("Gaussian kernel perf. based on matrix size")
plt.xlabel("matrix size")
plt.ylabel("time (s)");
plt.savefig("gaussian_performance.png")

In [ ]:
gauss_df

In [ ]:
speed_up_gauss_df = gauss_df.apply(lambda x: gauss_df['numpy'] / x)

In [ ]:
speed_up_gauss_df

In [ ]:
# extracting results pandas data frame
gauss_df.to_csv("./gauss_df.csv")

In [ ]:
speed_up_gauss_df[["numpy", "jit"]].plot.bar(rot=0)
plt.title("speedup of gaussian kernel compared to numpy version")
plt.xlabel("matrix size")
plt.ylabel("speedup");
plt.savefig("gaussian_speedup.png")